In [3]:
#%cd C:/Users/Mathieu/Desktop/Projets/Benter
%cd /home/mathieu/Mindsay/mathieu/Benter-Project

/home/mathieu/Mindsay/mathieu/Benter-Project


In [4]:
from models.horse import Horse
from database.setup import create_sqlalchemy_session
import tqdm
import re

from utils.music import parse_music

BATCH_SIZE = 1e5

In [7]:
start_id=0
with create_sqlalchemy_session() as db_session:
    wrong_horse_ids=set()

    horses=db_session.query(Horse).filter(Horse.id>start_id).order_by(Horse.id.asc()).limit(BATCH_SIZE).all()
    for horse in tqdm.tqdm(horses, desc=f'{start_id}', total=BATCH_SIZE):
        ages = [runner.age for runner in horse.runners if runner.age is not None]
        if sorted(ages) != ages:
            wrong_horse_ids.add(horse.id)
            
    while len(horses)==BATCH_SIZE:
        start_id=horses[-1].id
        horses=db_session.query(Horse).filter(Horse.id>start_id).order_by(Horse.id.asc()).limit(BATCH_SIZE).all()
        for horse in tqdm.tqdm(horses, desc=f'{start_id}', total=len(horses)):
            ages = [runner.age for runner in horse.runners if runner.age is not None]
            if sorted(ages) != ages:
                wrong_horse_ids.add(horse.id)

0: 100%|██████████| 10000/10000.0 [00:09<00:00, 1061.27it/s]
10000: 100%|██████████| 10000/10000.0 [00:07<00:00, 1304.66it/s]
20000: 100%|██████████| 10000/10000.0 [00:07<00:00, 1347.80it/s]
30000: 100%|██████████| 10000/10000.0 [00:07<00:00, 1382.60it/s]
40000: 100%|██████████| 10000/10000.0 [00:07<00:00, 1281.87it/s]
50000: 100%|██████████| 10000/10000.0 [00:06<00:00, 1444.00it/s]
60000: 100%|██████████| 10000/10000.0 [00:07<00:00, 1338.15it/s]
70000: 100%|██████████| 10000/10000.0 [00:07<00:00, 1371.43it/s]
80000: 100%|██████████| 10000/10000.0 [00:06<00:00, 1433.18it/s]
90000: 100%|██████████| 10000/10000.0 [00:06<00:00, 1465.28it/s]
100000: 100%|██████████| 10000/10000.0 [00:06<00:00, 1476.34it/s]
110000: 100%|██████████| 10000/10000.0 [00:06<00:00, 1447.83it/s]
120000: 100%|██████████| 10000/10000.0 [00:08<00:00, 1182.50it/s]
130000: 100%|██████████| 10000/10000.0 [00:07<00:00, 1256.47it/s]
140000: 100%|██████████| 10000/10000.0 [00:08<00:00, 1144.27it/s]
150000: 100%|██████████|

{131072,
 2051,
 18437,
 88071,
 24585,
 251913,
 286729,
 200716,
 10253,
 198669,
 292877,
 38929,
 16403,
 135188,
 290838,
 24599,
 122904,
 135191,
 53274,
 137240,
 30748,
 79902,
 79903,
 131104,
 221215,
 22562,
 245796,
 75819,
 290860,
 112685,
 192557,
 221232,
 141364,
 16437,
 180276,
 213045,
 114748,
 223292,
 63550,
 49215,
 67647,
 100415,
 110659,
 69,
 30789,
 26695,
 176199,
 196679,
 32842,
 36938,
 219212,
 243785,
 262221,
 196692,
 4181,
 174165,
 26718,
 272482,
 79973,
 127078,
 36967,
 75879,
 79976,
 153703,
 194662,
 65644,
 4206,
 6257,
 4210,
 180339,
 65655,
 163959,
 198779,
 276603,
 67712,
 14466,
 57475,
 92291,
 26757,
 321674,
 6283,
 16526,
 297102,
 30865,
 174226,
 155795,
 2197,
 12438,
 28822,
 49305,
 71835,
 125083,
 47262,
 2207,
 34976,
 174238,
 67746,
 53411,
 170146,
 67749,
 77991,
 30888,
 80042,
 196781,
 118958,
 4273,
 166066,
 24755,
 159923,
 12471,
 71866,
 98490,
 28864,
 12482,
 10439,
 102601,
 192715,
 159948,
 162006,
 8407

In [8]:
len(wrong_horse_ids)

1101

In [9]:
162684 in wrong_horse_ids

False

In [6]:
start_id=0
with create_sqlalchemy_session() as db_session:

    horses=db_session.query(Horse).filter(Horse.id>start_id).order_by(Horse.id.asc()).limit(BATCH_SIZE).all()
    
    for horse in tqdm.tqdm(horses, desc=f'{start_id}', total=BATCH_SIZE):
        origins={runner.origins for runner in horse.runners if runner.origins is not None}
        if not origins:
            continue
        if len(origins)==2:
            continue
        print(origins, horse.id)
        break

0:   0%|          | 0/100000.0 [00:00<?, ?it/s]

{'Turgeon-Shannon'} 1


In [7]:
a="Turgeon-Shannon"

In [10]:
father_name, mother_name=re.split("[-/]",a)

In [12]:
a.upper()

'TURGEON-SHANNON'

In [11]:
father_name=father_name_name.upper()
mother_name=mother_name.upper()

NameError: name 'father_name_name' is not defined

In [39]:
from collections import namedtuple
from typing import List

MusicEvent = namedtuple("MusicEvent", ["rank", "race_type", "year"])


def parse_unibet_music(current_year:int, ) ->List[MusicEvent]:

    music_events:List[MusicEvent]=[]
    events=re.findall(r"(([ATRDN]|\d)([a-z]))|(\((\d{1,2})\))", runner.music)
    for event in events:
        race_event, rank, race_type, _, year = event
        if year:
            assert not race_event
            next_current_year = int(f'20{year}')
            # We read music from left to right to go backward in time
            assert next_current_year < current_year
            current_year=next_current_year
            continue
        assert not year
        music_events.append(MusicEvent(rank=rank, race_type=race_type, year=current_year))
        
    return music_events
    

In [40]:
music_events

[MusicEvent(rank='4', race_type='o', year=2008),
 MusicEvent(rank='8', race_type='o', year=2007),
 MusicEvent(rank='2', race_type='o', year=2007),
 MusicEvent(rank='3', race_type='o', year=2005),
 MusicEvent(rank='3', race_type='o', year=2005),
 MusicEvent(rank='6', race_type='o', year=2005),
 MusicEvent(rank='5', race_type='o', year=2005),
 MusicEvent(rank='6', race_type='o', year=2005),
 MusicEvent(rank='1', race_type='o', year=2005),
 MusicEvent(rank='4', race_type='o', year=2004),
 MusicEvent(rank='2', race_type='o', year=2004),
 MusicEvent(rank='1', race_type='o', year=2004)]

In [36]:
runner.music

'4o (07) 8o 2o (05) 3o 3o 6o 5o 6o 1o (04) 4o 2o 1o'

In [26]:
race_type

'o'

In [27]:
_

''

In [28]:
year

''

In [29]:
race_type

'o'